In [1]:
from matplotlib import pyplot as plt
import cv2
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
import numpy as np
import imutils
import cv2
from skimage.morphology import remove_small_objects
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.keras import backend as K
from scipy.ndimage import center_of_mass

2023-11-07 14:16:01.847529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
image = 'Photos/20200829_192717.png'
# image = 'Photos/whiteout.png'
# image = 'Photos/Screenshot 2020-09-09 at 17.34.16.png'
im = cv2.imread(image)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(im)

In [ ]:
im.shape

In [ ]:
def print_img(im, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.imshow(im, cmap='gray')

def crop_img(img, scale=1.0):
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * scale, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    return img_cropped

def find_puzzle(image, debug=False):
    # convert the image to grayscale and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (21, 21), 4)
        # apply adaptive thresholding and then invert the threshold map
    thresh = cv2.adaptiveThreshold(
        blurred, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    thresh = cv2.bitwise_not(thresh)
    # check to see if we are visualizing each step of the image
    # processing pipeline (in this case, thresholding)
    if debug: 
        print_img(thresh)
    dilated = cv2.dilate(thresh, kernel = np.ones((1,1),np.uint8), iterations=4)
    if debug: 
        print_img(dilated)
    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # initialize a contour that corresponds to the puzzle outline
    puzzleCnt = []
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if our approximated contour has four points, then we can
        # assume we have found the outline of the puzzle
        if len(approx) == 4:
            puzzleCnt.append(approx)
            break
        # if the puzzle contour is empty then our script could not find
    if puzzleCnt is None:
        raise Exception(("Could not find Slitherlinker puzzle outline. "
            "Try debugging your thresholding and contour steps."))
    # check to see if we are visualizing the outline of the detected
    # Sudoku puzzle
    if debug:
        # draw the contour of the puzzle on the image and then display
        # it to our screen for visualization/debugging purposes
        output = image.copy()
        cv2.drawContours(output, puzzleCnt, -1, (0, 255, 0), 2)
        print_img(output)
    puzzle = four_point_transform(image, puzzleCnt[0].reshape(4, 2))
    warped = four_point_transform(gray, puzzleCnt[0].reshape(4, 2))
    if debug:
        print_img(warped)
    return puzzle, warped

In [ ]:
puzzle, warped = find_puzzle(im, debug=True)

In [ ]:
# gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# print_img(gray)

In [ ]:
# blurred = cv2.GaussianBlur(gray, (15,15), 4)
# # blurred = cv2.blur(gray, (15, 15), 3)
#     # apply adaptive thresholding and then invert the threshold map
# print_img(blurred)

In [ ]:
# thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
# print_img(thresh)

In [ ]:
# thresh = cv2.bitwise_not(thresh)
# print_img(thresh)

In [ ]:
# dilated = cv2.dilate(thresh, kernel = np.ones((1,1),np.uint8), iterations=4)
# print_img(dilated)

In [ ]:
# cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = imutils.grab_contours(cnts)
# cnts = sorted(cnts, key=cv2.contourArea, reverse=True)


In [ ]:
# all_contours = im.copy()
# cv2.drawContours(all_contours, cnts, -1, (0, 255, 0), 2)
# print_img(all_contours)

In [ ]:
# # initialize a contour that corresponds to the puzzle outline
# puzzleCnt = []
# # loop over the contours
# for c in cnts:
#     # approximate the contour
#     peri = cv2.arcLength(c, True)
#     approx = cv2.approxPolyDP(c, 0.02 * peri, True)
#     # if our approximated contour has four points, then we can
#     # assume we have found the outline of the puzzle
#     if len(approx) == 4:
#         puzzleCnt.append(approx)
#         break
# len(puzzleCnt)

In [ ]:
# output = im.copy()
# cv2.drawContours(output, puzzleCnt, -1, (0, 255, 0), 2)
# print_img(output)

In [ ]:
# puzzle = four_point_transform(im, puzzleCnt[0].reshape(4, 2))
# warped = four_point_transform(gray, puzzleCnt[0].reshape(4, 2))

In [ ]:
puzzle = crop_img(puzzle, 0.93)
warped = crop_img(warped, 0.93)
print_img(puzzle, figsize=(5,5))
print_img(warped, figsize=(5,5))

In [ ]:
print(warped.shape)
w, h = warped.shape
cell_w, cell_h = int(w / 6), int(h / 6)
min_size = int(cell_w * cell_h * 0.2)

In [ ]:
indices = []
fig, axs = plt.subplots(6,6, figsize=(10,10))
for i in range(6):
    for j in range(6):
        sub_img = warped[cell_w * i :cell_w * (i + 1), cell_h * j: cell_h * (j + 1)]
        cleaned = remove_small_objects(~(sub_img > 128), min_size=(cell_w * cell_h * 0.01))
#         cleaned = sub_img > 128
#         cleaned = clear_border(cleaned)
        if np.sum(cleaned) > 0:
            axs[i,j].imshow(cleaned)
            indices.append((i,j))

In [ ]:
def reduce_to_4_classes(x, y):
    res_x, res_y = [], []
    for x, y in zip(x, y):
        if y < 4:
            res_x.append(x)
            res_y.append(y)
    return np.array(res_x), np.array(res_y)
    
def translate_to_com(im, com):
    x_trans = int(im.shape[0]//2-com[0])
    y_trans = int(im.shape[1]//2-com[1])

    # Pad and remove pixels from image to perform translation

    if x_trans > 0:
        im2 = np.pad(im, ((x_trans, 0), (0, 0)), mode='constant')
        im2 = im2[:im.shape[0]-x_trans, :]
    else:
        im2 = np.pad(im, ((0, -x_trans), (0, 0)), mode='constant')
        im2 = im2[-x_trans:, :]

    if y_trans > 0:
        im3 = np.pad(im2, ((0, 0), (y_trans, 0)), mode='constant')
        im3 = im3[:, :im.shape[0]-y_trans]

    else:
        im3 = np.pad(im2, ((0, 0), (0, -y_trans)), mode='constant')
        im3 = im3[:, -y_trans:]
        
    return im3


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
# Model / data parameters
num_classes = 4
input_shape = (28, 28, 1)

datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     brightness_range=(0.8, 1.4),
    zoom_range=(0.8, 1.6),
    shear_range=0.3,
    fill_mode='nearest',
    validation_split=0.1,
#     horizontal_flip=True
)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


In [ ]:
if False and os.path.exists('model.hd5'):
    model = keras.models.load_model('model.hd5')
else:
    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train, y_train = reduce_to_4_classes(x_train, y_train)
    x_test, y_test = reduce_to_4_classes(x_test, y_test)
    
    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    print("x_train shape:", x_train.shape)
    print(x_train.shape[0], "train samples")
    print(x_test.shape[0], "test samples")
    # convert class vectors to binary class matrices
    
    y_train = keras.utils.to_categorical(y_train, 4)
    y_test = keras.utils.to_categorical(y_test, 4)

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy", f1_m])

    batch_size = 8
    epochs = 60
    
    model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), validation_data=(x_test, y_test), epochs=epochs,
              callbacks=[keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        mode="auto",
        restore_best_weights=True,)]
    )

In [ ]:
fig, axs = plt.subplots(6,6, figsize=(10, 10))
for i, j in indices:
    offset_x = 0
    offset_y = 0
    sub_img = warped[offset_x + cell_w * i: offset_x + cell_w * (i +1), offset_y + cell_h * j: offset_y + cell_h * (j + 1)]
    sub_img = cv2.GaussianBlur(sub_img, (11, 11), 2)
    sub_img = ~(sub_img > 128)

    sub_img = remove_small_objects(sub_img, min_size=(cell_w * cell_h * 0.01)) * 255
    com = center_of_mass(sub_img)
    sub_img = translate_to_com(sub_img, com)
    sub_img = cv2.resize(sub_img.astype(np.uint8), dsize=(28,28))
    
    sub_img = sub_img.reshape(input_shape)
    axs[i][j].imshow(sub_img / 255.)
    pred_vector = model.predict(np.array([sub_img > 0]))[0]
    prediction = np.argmax(pred_vector)
    print(f'Prediction ({i,j}): {prediction} ({pred_vector[prediction]})')

In [ ]:
model.save_weights('model.hd5')